In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import src.config as config

In [3]:
from datetime import datetime, timedelta

import pandas as pd

current_date = pd.to_datetime(datetime.utcnow()).floor('H')
print(f'{current_date=}')

# we fetch raw data for the last 28 days, to add redundancy to our data pipeline
fetch_data_to = current_date
fetch_data_from = current_date - timedelta(days=70) #28

current_date=Timestamp('2025-03-06 09:00:00')


In [4]:
from src.data import load_raw_data

def fetch_batch_raw_data(from_date: datetime, to_date: datetime) -> pd.DataFrame:
    """
    Simulate production data by sampling historical data from 52 weeks ago (i.e. 1 year)
    """
    from_date_ = from_date - timedelta(days=7*52)
    to_date_ = to_date - timedelta(days=7*52)
    print(f'{from_date=}, {to_date_=}')

    # # download 2 files from website
    # rides = load_raw_data(year=from_date_.year) #, months=from_date_.month)
    # rides = rides[(rides.pickup_datetime >= from_date_) & (rides.pickup_datetime < to_date_)]
   
    # rides_2 = load_raw_data(year=to_date_.year) #, months=to_date_.month)
    # rides_2 = rides_2[(rides_2.pickup_datetime < to_date_) & (rides_2.pickup_datetime < to_date_)]

    # rides = pd.concat([rides, rides_2]) 

    # Intenta cargar los datos del primer año
    rides = load_raw_data(year=from_date_.year)
    rides = rides[(rides.pickup_datetime >= from_date_) & (rides.pickup_datetime < to_date_)]


    ### FIX 1 ----- Esto hago que no se ejecute para probar si es el error
    # # Verifica si los años son diferentes
    # if from_date_.year != to_date_.year:
    # # Carga los datos del segundo año
    #     rides_2 = load_raw_data(year=to_date_.year)
    #     rides_2 = rides_2[(rides_2.pickup_datetime >= from_date_) & (rides_2.pickup_datetime < to_date_)]
    #     rides = pd.concat([rides, rides_2]) 


    # shift the data to pretend this is recent data
    rides['pickup_datetime'] += timedelta(days=7*52)

    rides.sort_values(by=['pickup_location_id', 'pickup_datetime'], inplace=True)

    return rides

In [5]:
rides = fetch_batch_raw_data(from_date=fetch_data_from, to_date=fetch_data_to)

from_date=Timestamp('2024-12-26 09:00:00'), to_date_=Timestamp('2024-03-07 09:00:00')
File 2023 was already in local storage
File 2024 was already in local storage


In [7]:
len(rides.pickup_location_id.unique())

356

In [8]:
from src.data import transform_raw_data_into_ts_data
ts_data = transform_raw_data_into_ts_data(rides)

100%|██████████| 356/356 [00:02<00:00, 176.17it/s]


In [9]:
rides.head()

,pickup_datetime,pickup_location_id,pickup_hour
1730583,2024-10-22 08:10:13,2,2024-10-22 08:00:00
1897291,2024-10-22 08:26:54,2,2024-10-22 08:00:00
1836014,2024-10-22 08:29:18,2,2024-10-22 08:00:00
1916165,2024-10-22 08:35:43,2,2024-10-22 08:00:00
1921488,2024-10-22 08:38:11,2,2024-10-22 08:00:00


In [10]:
ts_data.tail()

,pickup_hour,rides,pickup_location_id
598075,2024-12-31 02:00:00,0,362
598076,2024-12-31 03:00:00,0,362
598077,2024-12-31 04:00:00,0,362
598078,2024-12-31 05:00:00,0,362
598079,2024-12-31 06:00:00,0,362


In [11]:
import hopsworks

# connect to the project
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

# connect to the feature store
feature_store = project.get_feature_store()

# connect to the feature group
feature_group = feature_store.get_or_create_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
    description="Time-series data at hourly frequency",
    primary_key = ['pickup_location_id', 'pickup_hour'],
    event_time='pickup_hour',
)

2024-12-31 08:19:29,320 INFO: Initializing external client
2024-12-31 08:19:29,323 INFO: Base URL: https://c.app.hopsworks.ai:443
2024-12-31 08:19:34,011 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/100501


In [12]:
feature_group.insert(ts_data, write_options={"wait_for_job": True})

Uploading Dataframe: 100.00% |██████████| Rows 530880/530880 | Elapsed Time: 03:18 | Remaining Time: 00:00


Launching job: time_series_hourly_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/100501/jobs/named/time_series_hourly_feature_group_1_offline_fg_materialization/executions
2024-12-31 08:23:09,205 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2024-12-31 08:23:15,859 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2024-12-31 08:26:32,549 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2024-12-31 08:26:32,685 INFO: Waiting for log aggregation to finish.
2024-12-31 08:27:16,560 INFO: Execution finished successfully.


(Job('time_series_hourly_feature_group_1_offline_fg_materialization', 'SPARK'),
 None)